# Tipos de Encoding - Codificação

* Tabela ASCCII não contém caracteres especiais contruida para o idioma inglês.

ISO 8859-1 ou mais mais conhecido como latin-1

ISO/IEC 8859-1 é uma codificação de caracteres do alfabeto latino, a primeira parte da ISO 8859. Foi desenvolvida pela ISO, e posteriormente passou a ser mantida pela ISO e pela IEC. Este padrão é a base de outros mapeamentos amplamente usados como o ISO-8859-1 e o Windows-1252

* https://docs.python.org/3/library/codecs.html#standard-encodings

* https://upload.wikimedia.org/wikipedia/commons/d/dd/ASCII-Table.svg

# Usando for para leitura de arquivo

* Usar o for consome menos memória que o método readlines ou readline.

# Um pouco mais sobre o with

Utilizamos o comando with para gerenciar o contexto de utilização do arquivo. Além de arquivos, podemos utilizar o with para gerenciar processos que precisam de uma pré e pós condição de execução; por exemplo: abrir e fechar o arquivo, realizar conexão com o banco de dados, sockets, entre outros.

O objeto que está sendo manipulado pelo with precisa implementar dois métodos mágicos: __enter__() e __exit__().

O método __enter__() é executado logo no início da chamada da função e retorna uma representação do objeto que está sendo executada no contexto (ou context guard). Ao final, o método __exit__() é invocado, e o contexto da execução, finalizado.

Para saber um pouco mais sobre o statement, leia este post do blog preshing.com (em inglês).

* https://preshing.com/20110920/the-python-with-statement-by-example/

# Otimização de memória

O Python utiliza um buffer, uma região temporária na memória, como área de transferência para os dados que estão sendo lidos ou escritos em um arquivo.

Quando lemos um arquivo, por exemplo, o Python pega os dados do arquivo, passa para o buffer, e, então, o conteúdo do arquivo pode ser utilizado pela aplicação.

Quando iteramos sobre um arquivo, o Python vai colocando os dados no buffer à medida que o loop é executado, sendo uma boa forma de lidar com a memória consumida pelo sistema.

Por que é interessante não carregar todo o conteúdo do arquivo na memória de uma só vez?

* Caso o arquivo seja muito grande, pode ocorrer de não existir espaço suficiente na memória para alocar todos os dados, ocasionando um erro de memória.

# Segurança com arquivos pickle

Neste post do blog da Synopsys https://www.synopsys.com/blogs/software-security/python-pickling/  (em inglês), é tratado um pouco sobre o pickle e como usá-lo de forma segura, caso estejam interessados, é recomendo a leitura.

# DAOs para acesso a dados

No vídeo, criamos um arquivo utils para realizar a conversão dos dados para JSON e para Pickle.

Pensando em uma abordagem mais orientada a objetos, podemos utilizar um objeto de acesso a dados que fica sendo responsável pela comunicação do mundo Python com o mundo dos arquivos. Este objeto é conhecido como DAO, ou Objeto de Acesso a Dados (Data Access Object, em inglês).

Quem já trabalhou com banco de dados provavelmente conhece esse padrão de persistência. O DAO é um padrão de projeto muito utilizado por quem busca um meio de acessar seus dados. Popularmente, ele é muito utilizado para acessar o banco de dados e realizar as operações de criação, busca, exclusão e atualização. Além disso, ele pode ser utilizado para salvar e recuperar dados em arquivos, por exemplo.

Como podemos definir um DAO no projeto?

No nosso caso, queremos ler os contatos de um CSV e salvá-los em dois formatos, pickle e JSON. Pensando em isolar as responsabilidades, podemos definir três DAOs diferentes, um para cada tipo de arquivo. A fim de manter todos os objetos de acesso a dados com as mesmas assinaturas, podemos ainda definir uma classe abstrata que será estendida pelas classes que implementam de fato o acesso a dados.

Para criar uma classe abstrata, precisamos importar do pacote abc a classe ABC – que será herdada pela classe que padronizará os DAOs – e o decorator abstractmethod – que será usado para definir os métodos abstratos que serão implementados pelas classes filhas, portanto:

In [ ]:
from abc import ABC, abstractmethod

A classe que definirá as assinaturas das funções se chamará ContatoDao, e sua definição será parecida com esta:

In [ ]:
class ContatoDao(ABC):

    @abstractmethod
    def buscar_todos(self, caminho):
        pass

    @abstractmethod
    def salvar(self, contatos, caminho):
        pass

Agora, basta definirmos as implementações para essa classe, por exemplo, a classe que salva os dados em JSON pode ficar parecida com esta:

In [ ]:
class ContatoDaoJSON(ContatoDao):

    @abstractmethod
    def buscar_todos(self, caminho):
        contatos = []
        with open(caminho, mode='r') as arquivo:
            contatos_json = json.load(arquivo)
            for contato in contatos_json:
                        c = Contato(**contato)
                        contatos.append(c)

        return contatos   

    @abstractmethod
    def salvar(self, contatos, caminho):
        with open(caminho, mode='w') as arquivo:
    json.dump(contatos, arquivo, default=lambda objeto: objeto.__dict__) 


De uma forma análoga podemos ter as classes ContatoDaoPickle, ContatoDaoCSV, ContatoDaoSQL e as mais diversas implementações da classe ContatoDao, cada uma isolando a lógica de acesso em seu domínio  